In [1]:
from ray import tune
import ray
import copy
import yaml
import train_and_eval
import glob
import json
ray.init(log_to_driver=False)

/home/rah150030/.conda/envs/gpn2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rah150030/.conda/envs/gpn2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2022-10-24 15:54:01,231	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.0.0


In [2]:
def config_helper(config):
    return train_and_eval.run_experiment(**config)

In [3]:
with open('configs/gpn/ood_loc_gpn_16.yaml') as f:
    config = yaml.safe_load(f)

In [4]:
config['model']['latent_dist_reg'] = tune.grid_search([10 ** i for i in range(-8, 1)])
config['model']['KNN_K'] = tune.grid_search([5, 10, 25])
config['model']['normalize_dist_reg'] = tune.grid_search([False, ])
config['model']['dist_sigma'] = tune.grid_search([10 ** i for i in range(-3, 1)])

In [5]:
# dirs = sorted(glob.glob("../ray_results/ray_helper_2022*"))
# trials = glob.glob(dirs[-1] + "/*")

In [6]:
# rocs = []
# for a_trial in trials:
#     if ".json" in a_trial:
#         continue
#     with open(a_trial + "/result.json", 'r') as f:
#         try:
#             a_result_file = json.load(f)
#         except:
#             continue
#     rocs.append(a_result_file['test_ood_detection_epistemic_auroc'])
# max(rocs)

In [7]:
results = tune.run(
    config_helper,
    metric='test_ood_detection_epistemic_auroc',
    mode="max",
    num_samples=1,
    max_concurrent_trials=8,
    resources_per_trial={"gpu": 0.6},
    config=config,
    max_failures=1)

2022-10-24 15:54:10,822	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,model/KNN_K,model/dist_sigma,model/latent_dist...,model/normalize_d...
config_helper_02822_00000,RUNNING,10.176.130.19:4007448,5,0.001,1e-08,False
config_helper_02822_00001,RUNNING,10.176.130.19:4007508,10,0.001,1e-08,False
config_helper_02822_00002,RUNNING,10.176.130.19:4007571,25,0.001,1e-08,False
config_helper_02822_00003,RUNNING,10.176.130.19:4007626,5,0.01,1e-08,False
config_helper_02822_00004,RUNNING,10.176.130.19:4007681,10,0.01,1e-08,False
config_helper_02822_00005,RUNNING,10.176.130.19:4007737,25,0.01,1e-08,False
config_helper_02822_00006,RUNNING,10.176.130.19:4007891,5,0.1,1e-08,False
config_helper_02822_00007,RUNNING,10.176.130.19:4007946,10,0.1,1e-08,False
